In [326]:
import numpy as np
from scipy import sparse
import pandas as pd
import xgboost as xgb
import re
import string
import time
import seaborn as sns
import itertools
# import lightgbm as lgb - will be used in following weeks
# from bayes_opt import BayesianOptimization - will be used in following weeks
import seaborn as sns
import matplotlib.pyplot as plt

import random
from sklearn import preprocessing, pipeline, metrics, model_selection
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.preprocessing import Imputer
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.linear_model import LogisticRegression,RidgeClassifier
from sklearn.neural_network import MLPClassifier
from IPython.display import display
import datetime

%matplotlib inline 



In [327]:
train_data = pd.read_csv('../input/air_visit_data.csv')
air_store_info = pd.read_csv('../input/air_store_info.csv')
hpg_store_info = pd.read_csv('../input/hpg_store_info.csv')
air_reserve = pd.read_csv('../input/air_reserve.csv')
hpg_reserve = pd.read_csv('../input/hpg_reserve.csv')
store_id_relation = pd.read_csv('../input/store_id_relation.csv')
test_data = pd.read_csv('../input/sample_submission.csv')
date_info = pd.read_csv('../input/date_info.csv').rename(columns={'calendar_date':'visit_date'})
train_size = train_data.shape[0]

In [328]:
display(train_data.head())
display(test_data.head())

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0


In [329]:
test_data['visit_date'] = test_data['id'].map(lambda x: str(x).split('_')[2])
test_data['air_store_id'] = test_data['id'].map(lambda x: '_'.join(x.split('_')[:2]))
test_data.head()

,id,visitors,visit_date,air_store_id
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9


In [330]:
full_data = pd.concat([train_data,test_data])
display(full_data.head())
display(full_data.tail())

,air_store_id,id,visit_date,visitors
0,air_ba937bf13d40fb24,NaN,2016-01-13,25
1,air_ba937bf13d40fb24,NaN,2016-01-14,32
2,air_ba937bf13d40fb24,NaN,2016-01-15,29
3,air_ba937bf13d40fb24,NaN,2016-01-16,22
4,air_ba937bf13d40fb24,NaN,2016-01-18,6


,air_store_id,id,visit_date,visitors
32014,air_fff68b929994bfbd,air_fff68b929994bfbd_2017-05-27,2017-05-27,0
32015,air_fff68b929994bfbd,air_fff68b929994bfbd_2017-05-28,2017-05-28,0
32016,air_fff68b929994bfbd,air_fff68b929994bfbd_2017-05-29,2017-05-29,0
32017,air_fff68b929994bfbd,air_fff68b929994bfbd_2017-05-30,2017-05-30,0
32018,air_fff68b929994bfbd,air_fff68b929994bfbd_2017-05-31,2017-05-31,0


In [331]:
full_data['visit_date'] = pd.to_datetime(full_data['visit_date'])

In [332]:
full_data['visit_date'].head(2)

0   2016-01-13
1   2016-01-14
Name: visit_date, dtype: datetime64[ns]

In [333]:
full_data['dow'] = full_data['visit_date'].dt.dayofweek

In [334]:
full_data['dow'].head(3)

0    2
1    3
2    4
Name: dow, dtype: int64

In [335]:
full_data['year'] = full_data['visit_date'].dt.year

In [336]:
full_data['year'].head(3)

0    2016
1    2016
2    2016
Name: year, dtype: int64

In [337]:
full_data['month'] = full_data['visit_date'].dt.month
full_data['doy'] = full_data['visit_date'].dt.dayofyear
full_data['dom'] = full_data['visit_date'].dt.days_in_month
full_data['woy'] = full_data['visit_date'].dt.weekofyear

In [338]:
full_data['is_month_end'] = full_data['visit_date'].dt.is_month_end

DatetimeIndex.is_month_end
Logical indicating if last day of month (defined by frequency)

In [339]:
full_data['is_month_end'].head(2)

0    False
1    False
Name: is_month_end, dtype: bool

In [340]:
full_data['visit_date'] = full_data['visit_date'].dt.date

In [341]:
full_data['date_int'] = full_data['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)

In [342]:
full_data['date_int'].head(2)

0    20160113
1    20160114
Name: date_int, dtype: int64

In [343]:
air_store_info.air_area_name.head(4)

0    Hyōgo-ken Kōbe-shi Kumoidōri
1    Hyōgo-ken Kōbe-shi Kumoidōri
2    Hyōgo-ken Kōbe-shi Kumoidōri
3    Hyōgo-ken Kōbe-shi Kumoidōri
Name: air_area_name, dtype: object

In [344]:
air_store_info.head(4)

,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852


Store information

In [345]:
air_store_info['air_area_lv1'] = air_store_info.air_area_name.apply(lambda x:x.split(' ')[0])
air_store_info['air_area_lv2'] = air_store_info.air_area_name.apply(lambda x:x.split(' ')[1])
air_store_info['air_area_lv3'] = air_store_info.air_area_name.apply(lambda x:x.split(' ')[2])



In [346]:
air_store_info['air_area_lv1'].head(2)

0    Hyōgo-ken
1    Hyōgo-ken
Name: air_area_lv1, dtype: object

In [347]:
air_store_info['air_area_lv2'].head(4)

0    Kōbe-shi
1    Kōbe-shi
2    Kōbe-shi
3    Kōbe-shi
Name: air_area_lv2, dtype: object

In [348]:
hpg_store_info['hpg_area_lv1'] = hpg_store_info.hpg_area_name.apply(lambda x:x.split(' ')[0])
hpg_store_info['hpg_area_lv2'] = hpg_store_info.hpg_area_name.apply(lambda x:x.split(' ')[1])
hpg_store_info['hpg_area_lv3'] = hpg_store_info.hpg_area_name.apply(lambda x:x.split(' ')[2])

Create features that based on different levels of area

In [349]:
air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby(['latitude','longitude']).air_store_id.count().
                                    reset_index().rename(columns={'air_store_id':'air_stores_on_same_addr'}),
                             how='left',
                             on=['latitude','longitude'])


In [350]:
air_store_info.head(4)

,air_store_id,air_genre_name,air_area_name,latitude,longitude,air_area_lv1,air_area_lv2,air_area_lv3,air_stores_on_same_addr
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri,17
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri,17
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri,17
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri,17


In [351]:
air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby('air_area_lv1').air_store_id.count().\
                                    reset_index().rename(columns={'air_store_id':'air_stores_lv1'}),
                             how='left',
                             on='air_area_lv1')



In [352]:
air_store_info.head(4)

,air_store_id,air_genre_name,air_area_name,latitude,longitude,air_area_lv1,air_area_lv2,air_area_lv3,air_stores_on_same_addr,air_stores_lv1
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri,17,57
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri,17,57
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri,17,57
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri,17,57


In [353]:
air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby(['air_area_lv1','air_area_lv2']).air_store_id.count().\
                                    reset_index().rename(columns={'air_store_id':'air_stores_lv2'}),
                             how='left',
                             on=['air_area_lv1','air_area_lv2'])



In [354]:
air_store_info.head(4)

,air_store_id,air_genre_name,air_area_name,latitude,longitude,air_area_lv1,air_area_lv2,air_area_lv3,air_stores_on_same_addr,air_stores_lv1,air_stores_lv2
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri,17,57,25
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri,17,57,25
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri,17,57,25
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri,17,57,25


In [355]:
air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby('air_area_lv1').latitude.mean().\
                                    reset_index().rename(columns={'latitude':'mean_lat_air_lv1'}),
                             how='left',
                             on='air_area_lv1')



In [356]:
air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby('air_area_lv1').latitude.max().\
                                    reset_index().rename(columns={'latitude':'max_lat_air_lv1'}),
                             how='left',
                             on='air_area_lv1')

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby('air_area_lv1').latitude.min().\
                                    reset_index().rename(columns={'latitude':'min_lat_air_lv1'}),
                             how='left',
                             on='air_area_lv1')


air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby('air_area_lv1').longitude.mean().\
                                    reset_index().rename(columns={'longitude':'mean_lon_air_lv1'}),
                             how='left',
                             on='air_area_lv1')

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby('air_area_lv1').longitude.max().\
                                    reset_index().rename(columns={'longitude':'max_lon_air_lv1'}),
                             how='left',
                             on='air_area_lv1')

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby('air_area_lv1').longitude.min().\
                                    reset_index().rename(columns={'longitude':'min_lon_air_lv1'}),
                             how='left',
                             on='air_area_lv1')

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby(['air_area_lv1','air_area_lv2']).latitude.mean().\
                                    reset_index().rename(columns={'latitude':'mean_lat_air_lv2'}),
                             how='left',
                             on=['air_area_lv1','air_area_lv2'])

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby(['air_area_lv1','air_area_lv2']).latitude.max().\
                                    reset_index().rename(columns={'latitude':'max_lat_air_lv2'}),
                             how='left',
                             on=['air_area_lv1','air_area_lv2'])

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby(['air_area_lv1','air_area_lv2']).latitude.min().\
                                    reset_index().rename(columns={'latitude':'min_lat_air_lv2'}),
                             how='left',
                             on=['air_area_lv1','air_area_lv2'])


air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby(['air_area_lv1','air_area_lv2']).longitude.mean().\
                                    reset_index().rename(columns={'longitude':'mean_lon_air_lv2'}),
                             how='left',
                             on=['air_area_lv1','air_area_lv2'])

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby(['air_area_lv1','air_area_lv2']).longitude.max().\
                                    reset_index().rename(columns={'longitude':'max_lon_air_lv2'}),
                             how='left',
                             on=['air_area_lv1','air_area_lv2'])

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby(['air_area_lv1','air_area_lv2']).longitude.min().\
                                    reset_index().rename(columns={'longitude':'min_lon_air_lv2'}),
                             how='left',
                             on=['air_area_lv1','air_area_lv2'])


hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby(['latitude','longitude']).hpg_store_id.count().\
                                    reset_index().rename(columns={'hpg_store_id':'hpg_stores_on_same_addr'}),
                             how='left',
                             on=['latitude','longitude'])


hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby('hpg_area_lv1').hpg_store_id.count().\
                                    reset_index().rename(columns={'hpg_store_id':'hpg_stores_lv1'}),
                             how='left',
                             on='hpg_area_lv1')

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby(['hpg_area_lv1','hpg_area_lv2']).hpg_store_id.count().\
                                    reset_index().rename(columns={'hpg_store_id':'hpg_stores_lv2'}),
                             how='left',
                             on=['hpg_area_lv1','hpg_area_lv2'])


hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby('hpg_area_lv1').latitude.mean().\
                                    reset_index().rename(columns={'latitude':'mean_lat_hpg_lv1'}),
                             how='left',
                             on='hpg_area_lv1')

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby('hpg_area_lv1').latitude.max().\
                                    reset_index().rename(columns={'latitude':'max_lat_hpg_lv1'}),
                             how='left',
                             on='hpg_area_lv1')

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby('hpg_area_lv1').latitude.min().\
                                    reset_index().rename(columns={'latitude':'min_lat_hpg_lv1'}),
                             how='left',
                             on='hpg_area_lv1')


hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby('hpg_area_lv1').longitude.mean().\
                                    reset_index().rename(columns={'longitude':'mean_lon_hpg_lv1'}),
                             how='left',
                             on='hpg_area_lv1')

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby('hpg_area_lv1').longitude.max().\
                                    reset_index().rename(columns={'longitude':'max_lon_hpg_lv1'}),
                             how='left',
                             on='hpg_area_lv1')

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby('hpg_area_lv1').longitude.min().\
                                    reset_index().rename(columns={'longitude':'min_lon_hpg_lv1'}),
                             how='left',
                             on='hpg_area_lv1')

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby(['hpg_area_lv1','hpg_area_lv2']).latitude.mean().\
                                    reset_index().rename(columns={'latitude':'mean_lat_hpg_lv2'}),
                             how='left',
                             on=['hpg_area_lv1','hpg_area_lv2'])

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby(['hpg_area_lv1','hpg_area_lv2']).latitude.max().\
                                    reset_index().rename(columns={'latitude':'max_lat_hpg_lv2'}),
                             how='left',
                             on=['hpg_area_lv1','hpg_area_lv2'])

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby(['hpg_area_lv1','hpg_area_lv2']).latitude.min().\
                                    reset_index().rename(columns={'latitude':'min_lat_hpg_lv2'}),
                             how='left',
                             on=['hpg_area_lv1','hpg_area_lv2'])


hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby(['hpg_area_lv1','hpg_area_lv2']).longitude.mean().\
                                    reset_index().rename(columns={'longitude':'mean_lon_hpg_lv2'}),
                             how='left',
                             on=['hpg_area_lv1','hpg_area_lv2'])

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby(['hpg_area_lv1','hpg_area_lv2']).longitude.max().\
                                    reset_index().rename(columns={'longitude':'max_lon_hpg_lv2'}),
                             how='left',
                             on=['hpg_area_lv1','hpg_area_lv2'])

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby(['hpg_area_lv1','hpg_area_lv2']).longitude.min().\
                                    reset_index().rename(columns={'longitude':'min_lon_hpg_lv2'}),
                             how='left',
                             on=['hpg_area_lv1','hpg_area_lv2'])

In [357]:
list(hpg_store_info.columns.values)

['hpg_store_id',
 'hpg_genre_name',
 'hpg_area_name',
 'latitude',
 'longitude',
 'hpg_area_lv1',
 'hpg_area_lv2',
 'hpg_area_lv3',
 'hpg_stores_on_same_addr',
 'hpg_stores_lv1',
 'hpg_stores_lv2',
 'mean_lat_hpg_lv1',
 'max_lat_hpg_lv1',
 'min_lat_hpg_lv1',
 'mean_lon_hpg_lv1',
 'max_lon_hpg_lv1',
 'min_lon_hpg_lv1',
 'mean_lat_hpg_lv2',
 'max_lat_hpg_lv2',
 'min_lat_hpg_lv2',
 'mean_lon_hpg_lv2',
 'max_lon_hpg_lv2',
 'min_lon_hpg_lv2']

In [358]:
list(air_store_info.columns.values)

['air_store_id',
 'air_genre_name',
 'air_area_name',
 'latitude',
 'longitude',
 'air_area_lv1',
 'air_area_lv2',
 'air_area_lv3',
 'air_stores_on_same_addr',
 'air_stores_lv1',
 'air_stores_lv2',
 'mean_lat_air_lv1',
 'max_lat_air_lv1',
 'min_lat_air_lv1',
 'mean_lon_air_lv1',
 'max_lon_air_lv1',
 'min_lon_air_lv1',
 'mean_lat_air_lv2',
 'max_lat_air_lv2',
 'min_lat_air_lv2',
 'mean_lon_air_lv2',
 'max_lon_air_lv2',
 'min_lon_air_lv2']

In [359]:
air_store_info = pd.merge(air_store_info, store_id_relation, how='left', on='air_store_id')
air_store_info = pd.merge(air_store_info, hpg_store_info, how='left', on='hpg_store_id')
air_store_info = air_store_info.rename(columns={'latitude_x':'latitude_air',
                             'longitude_x':'longitude_air',
                             'latitude_y':'latitude_hpg',
                             'longitude_y':'longitude_hpg'})

air_store_info.head(5)

,air_store_id,air_genre_name,air_area_name,latitude_air,longitude_air,air_area_lv1,air_area_lv2,air_area_lv3,air_stores_on_same_addr,air_stores_lv1,...,min_lat_hpg_lv1,mean_lon_hpg_lv1,max_lon_hpg_lv1,min_lon_hpg_lv1,mean_lat_hpg_lv2,max_lat_hpg_lv2,min_lat_hpg_lv2,mean_lon_hpg_lv2,max_lon_hpg_lv2,min_lon_hpg_lv2
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri,17,57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri,17,57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri,17,57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri,17,57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō-to,Minato-ku,Shibakōen,51,444,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [360]:
display(air_store_info.tail())

,air_store_id,air_genre_name,air_area_name,latitude_air,longitude_air,air_area_lv1,air_area_lv2,air_area_lv3,air_stores_on_same_addr,air_stores_lv1,...,min_lat_hpg_lv1,mean_lon_hpg_lv1,max_lon_hpg_lv1,min_lon_hpg_lv1,mean_lat_hpg_lv2,max_lat_hpg_lv2,min_lat_hpg_lv2,mean_lon_hpg_lv2,max_lon_hpg_lv2,min_lon_hpg_lv2
824,air_9bf595ef095572fb,International cuisine,Tōkyō-to Shibuya-ku Shibuya,35.661777,139.704051,Tōkyō-to,Shibuya-ku,Shibuya,58,444,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
825,air_764f71040a413d4d,Asian,Tōkyō-to Shibuya-ku Shibuya,35.661777,139.704051,Tōkyō-to,Shibuya-ku,Shibuya,58,444,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
826,air_10bbe8acd943d8f6,Asian,Tōkyō-to Shibuya-ku Shibuya,35.661777,139.704051,Tōkyō-to,Shibuya-ku,Shibuya,58,444,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
827,air_7514d90009613cd6,Karaoke/Party,Hokkaidō Sapporo-shi Minami 3 Jōnishi,43.055460,141.340956,Hokkaidō,Sapporo-shi,Minami,19,46,...,41.786764,141.588326,143.714585,140.743511,43.051394,43.098159,42.942601,141.358825,141.429051,141.351419
828,air_c8fe396d6c46275d,Karaoke/Party,Hokkaidō Sapporo-shi Minami 3 Jōnishi,43.055460,141.340956,Hokkaidō,Sapporo-shi,Minami,19,46,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [361]:
list(air_store_info.columns.values)

['air_store_id',
 'air_genre_name',
 'air_area_name',
 'latitude_air',
 'longitude_air',
 'air_area_lv1',
 'air_area_lv2',
 'air_area_lv3',
 'air_stores_on_same_addr',
 'air_stores_lv1',
 'air_stores_lv2',
 'mean_lat_air_lv1',
 'max_lat_air_lv1',
 'min_lat_air_lv1',
 'mean_lon_air_lv1',
 'max_lon_air_lv1',
 'min_lon_air_lv1',
 'mean_lat_air_lv2',
 'max_lat_air_lv2',
 'min_lat_air_lv2',
 'mean_lon_air_lv2',
 'max_lon_air_lv2',
 'min_lon_air_lv2',
 'hpg_store_id',
 'hpg_genre_name',
 'hpg_area_name',
 'latitude_hpg',
 'longitude_hpg',
 'hpg_area_lv1',
 'hpg_area_lv2',
 'hpg_area_lv3',
 'hpg_stores_on_same_addr',
 'hpg_stores_lv1',
 'hpg_stores_lv2',
 'mean_lat_hpg_lv1',
 'max_lat_hpg_lv1',
 'min_lat_hpg_lv1',
 'mean_lon_hpg_lv1',
 'max_lon_hpg_lv1',
 'min_lon_hpg_lv1',
 'mean_lat_hpg_lv2',
 'max_lat_hpg_lv2',
 'min_lat_hpg_lv2',
 'mean_lon_hpg_lv2',
 'max_lon_hpg_lv2',
 'min_lon_hpg_lv2']

In [362]:
air_store_info.loc[:0].values

array([['air_0f0cdeee6c9bf3d7', 'Italian/French',
        'Hyōgo-ken Kōbe-shi Kumoidōri', 34.6951242, 135.19785249999998,
        'Hyōgo-ken', 'Kōbe-shi', 'Kumoidōri', 17, 57, 25,
        34.741237138596496, 34.815149, 34.688241399999995,
        135.09745040701765, 135.4063757, 134.6853528, 34.700598416000005,
        34.7202277, 34.688241399999995, 135.213229196, 135.265455,
        135.18725369999999, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan]], dtype=object)

In [363]:

display(full_data.head())

,air_store_id,id,visit_date,visitors,dow,year,month,doy,dom,woy,is_month_end,date_int
0,air_ba937bf13d40fb24,NaN,2016-01-13,25,2,2016,1,13,31,2,False,20160113
1,air_ba937bf13d40fb24,NaN,2016-01-14,32,3,2016,1,14,31,2,False,20160114
2,air_ba937bf13d40fb24,NaN,2016-01-15,29,4,2016,1,15,31,2,False,20160115
3,air_ba937bf13d40fb24,NaN,2016-01-16,22,5,2016,1,16,31,2,False,20160116
4,air_ba937bf13d40fb24,NaN,2016-01-18,6,0,2016,1,18,31,3,False,20160118


In [364]:
full_data = pd.merge(full_data, air_store_info, how='left', on='air_store_id')

In [365]:

display(full_data.head())

,air_store_id,id,visit_date,visitors,dow,year,month,doy,dom,woy,...,min_lat_hpg_lv1,mean_lon_hpg_lv1,max_lon_hpg_lv1,min_lon_hpg_lv1,mean_lat_hpg_lv2,max_lat_hpg_lv2,min_lat_hpg_lv2,mean_lon_hpg_lv2,max_lon_hpg_lv2,min_lon_hpg_lv2
0,air_ba937bf13d40fb24,NaN,2016-01-13,25,2,2016,1,13,31,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,air_ba937bf13d40fb24,NaN,2016-01-14,32,3,2016,1,14,31,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,air_ba937bf13d40fb24,NaN,2016-01-15,29,4,2016,1,15,31,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,air_ba937bf13d40fb24,NaN,2016-01-16,22,5,2016,1,16,31,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,air_ba937bf13d40fb24,NaN,2016-01-18,6,0,2016,1,18,31,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [366]:
list(full_data.columns.values)

['air_store_id',
 'id',
 'visit_date',
 'visitors',
 'dow',
 'year',
 'month',
 'doy',
 'dom',
 'woy',
 'is_month_end',
 'date_int',
 'air_genre_name',
 'air_area_name',
 'latitude_air',
 'longitude_air',
 'air_area_lv1',
 'air_area_lv2',
 'air_area_lv3',
 'air_stores_on_same_addr',
 'air_stores_lv1',
 'air_stores_lv2',
 'mean_lat_air_lv1',
 'max_lat_air_lv1',
 'min_lat_air_lv1',
 'mean_lon_air_lv1',
 'max_lon_air_lv1',
 'min_lon_air_lv1',
 'mean_lat_air_lv2',
 'max_lat_air_lv2',
 'min_lat_air_lv2',
 'mean_lon_air_lv2',
 'max_lon_air_lv2',
 'min_lon_air_lv2',
 'hpg_store_id',
 'hpg_genre_name',
 'hpg_area_name',
 'latitude_hpg',
 'longitude_hpg',
 'hpg_area_lv1',
 'hpg_area_lv2',
 'hpg_area_lv3',
 'hpg_stores_on_same_addr',
 'hpg_stores_lv1',
 'hpg_stores_lv2',
 'mean_lat_hpg_lv1',
 'max_lat_hpg_lv1',
 'min_lat_hpg_lv1',
 'mean_lon_hpg_lv1',
 'max_lon_hpg_lv1',
 'min_lon_hpg_lv1',
 'mean_lat_hpg_lv2',
 'max_lat_hpg_lv2',
 'min_lat_hpg_lv2',
 'mean_lon_hpg_lv2',
 'max_lon_hpg_lv2',
 'mi

In [367]:
print ('before')
display(air_reserve.head())
display(hpg_reserve.head())



before


,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5


,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13


In [368]:
air_reserve['visit_date'] = air_reserve['visit_datetime'].apply(lambda x:x[:10])

In [369]:
air_reserve['visit_date'].head()

0    2016-01-01
1    2016-01-01
2    2016-01-01
3    2016-01-01
4    2016-01-01
Name: visit_date, dtype: object

In [370]:
air_reserve['reserve_date'] = air_reserve['reserve_datetime'].apply(lambda x:x[:10])

In [371]:
air_reserve['reserve_date'] .head()

0    2016-01-01
1    2016-01-01
2    2016-01-01
3    2016-01-01
4    2016-01-01
Name: reserve_date, dtype: object

In [372]:
air_reserve['reserve_datetime'] = pd.to_datetime(air_reserve['reserve_datetime'])

In [373]:
air_reserve['reserve_datetime'].head()

0   2016-01-01 16:00:00
1   2016-01-01 19:00:00
2   2016-01-01 19:00:00
3   2016-01-01 16:00:00
4   2016-01-01 01:00:00
Name: reserve_datetime, dtype: datetime64[ns]

In [374]:
air_reserve['reserve_date'] = air_reserve['reserve_datetime'].dt.date

In [375]:
air_reserve['reserve_date'].head()

0    2016-01-01
1    2016-01-01
2    2016-01-01
3    2016-01-01
4    2016-01-01
Name: reserve_date, dtype: object

In [376]:
air_reserve['visit_datetime'] = pd.to_datetime(air_reserve['visit_datetime'])
air_reserve['visit_date'] = air_reserve['visit_datetime'].dt.date



In [377]:
air_reserve['visit_datetime'].head()

0   2016-01-01 19:00:00
1   2016-01-01 19:00:00
2   2016-01-01 19:00:00
3   2016-01-01 20:00:00
4   2016-01-01 20:00:00
Name: visit_datetime, dtype: datetime64[ns]

In [378]:
air_reserve['visit_date'].head()

0    2016-01-01
1    2016-01-01
2    2016-01-01
3    2016-01-01
4    2016-01-01
Name: visit_date, dtype: object

In [379]:
air_reserve['reserve_datetime_diff'] = air_reserve.apply(lambda r: (r['visit_datetime'] - r['reserve_datetime']).seconds 
                                                         * r['reserve_visitors']/3600/24.0, axis=1)



In [380]:
hpg_reserve['visit_date'] = hpg_reserve['visit_datetime'].apply(lambda x:x[:10])
hpg_reserve['reserve_date'] = hpg_reserve['reserve_datetime'].apply(lambda x:x[:10])
hpg_reserve['reserve_datetime'] = pd.to_datetime(hpg_reserve['reserve_datetime'])
hpg_reserve['reserve_date'] = hpg_reserve['reserve_datetime'].dt.date
hpg_reserve['visit_datetime'] = pd.to_datetime(hpg_reserve['visit_datetime'])
hpg_reserve['visit_date'] = hpg_reserve['visit_datetime'].dt.date
hpg_reserve['reserve_datetime_diff'] = hpg_reserve.apply(lambda r: (r['visit_datetime'] - r['reserve_datetime']).seconds
                                                         * r['reserve_visitors']/3600/24.0, axis=1)



In [381]:
air_reserve.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors,visit_date,reserve_date,reserve_datetime_diff
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1,2016-01-01,2016-01-01,0.125000
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3,2016-01-01,2016-01-01,0.000000
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6,2016-01-01,2016-01-01,0.000000
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2,2016-01-01,2016-01-01,0.333333
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5,2016-01-01,2016-01-01,3.958333


In [382]:
hpg_reserve.head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors,visit_date,reserve_date,reserve_datetime_diff
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1,2016-01-01,2016-01-01,0.083333
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3,2016-01-01,2016-01-01,0.875000
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2,2016-01-01,2016-01-01,0.166667
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5,2016-01-01,2016-01-01,1.250000
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13,2016-01-01,2016-01-01,7.583333


In [383]:
list(hpg_reserve.columns.values)

['hpg_store_id',
 'visit_datetime',
 'reserve_datetime',
 'reserve_visitors',
 'visit_date',
 'reserve_date',
 'reserve_datetime_diff']

In [384]:
air_reserve_grp = air_reserve.groupby(['air_store_id','visit_date'])['reserve_visitors','reserve_datetime_diff'].\
            sum().reset_index().rename(columns={'reserve_visitors':'air_rvs',
                                               'reserve_datetime_diff':'air_rv_dt_diff'})

In [385]:
hpg_reserve_grp = hpg_reserve.groupby(['hpg_store_id','visit_date'])['reserve_visitors','reserve_datetime_diff'].\
            sum().reset_index().rename(columns={'reserve_visitors':'hpg_rvs',
                                               'reserve_datetime_diff':'hpg_rv_dt_diff'})

In [386]:
air_reserve_grp['mean_air_rv_dt_diff'] = air_reserve_grp.air_rv_dt_diff / air_reserve_grp.air_rvs
hpg_reserve_grp['mean_hpg_rv_dt_diff'] = hpg_reserve_grp.hpg_rv_dt_diff / hpg_reserve_grp.hpg_rvs    
    
display(air_reserve_grp.head())
display(hpg_reserve_grp.head()) 

,air_store_id,visit_date,air_rvs,air_rv_dt_diff,mean_air_rv_dt_diff
0,air_00a91d42b08b08d9,2016-10-31,2,0.333333,0.166667
1,air_00a91d42b08b08d9,2016-12-05,9,1.500000,0.166667
2,air_00a91d42b08b08d9,2016-12-14,18,6.750000,0.375000
3,air_00a91d42b08b08d9,2016-12-17,2,0.250000,0.125000
4,air_00a91d42b08b08d9,2016-12-20,4,0.500000,0.125000


,hpg_store_id,visit_date,hpg_rvs,hpg_rv_dt_diff,mean_hpg_rv_dt_diff
0,hpg_001112ef76b9802c,2016-02-26,9,2.250000,0.250000
1,hpg_001112ef76b9802c,2016-03-17,3,0.000000,0.000000
2,hpg_001112ef76b9802c,2016-03-31,5,4.791667,0.958333
3,hpg_001112ef76b9802c,2016-04-05,13,4.333333,0.333333
4,hpg_001112ef76b9802c,2016-04-18,9,1.875000,0.208333


In [387]:

list(full_data.columns.values)

['air_store_id',
 'id',
 'visit_date',
 'visitors',
 'dow',
 'year',
 'month',
 'doy',
 'dom',
 'woy',
 'is_month_end',
 'date_int',
 'air_genre_name',
 'air_area_name',
 'latitude_air',
 'longitude_air',
 'air_area_lv1',
 'air_area_lv2',
 'air_area_lv3',
 'air_stores_on_same_addr',
 'air_stores_lv1',
 'air_stores_lv2',
 'mean_lat_air_lv1',
 'max_lat_air_lv1',
 'min_lat_air_lv1',
 'mean_lon_air_lv1',
 'max_lon_air_lv1',
 'min_lon_air_lv1',
 'mean_lat_air_lv2',
 'max_lat_air_lv2',
 'min_lat_air_lv2',
 'mean_lon_air_lv2',
 'max_lon_air_lv2',
 'min_lon_air_lv2',
 'hpg_store_id',
 'hpg_genre_name',
 'hpg_area_name',
 'latitude_hpg',
 'longitude_hpg',
 'hpg_area_lv1',
 'hpg_area_lv2',
 'hpg_area_lv3',
 'hpg_stores_on_same_addr',
 'hpg_stores_lv1',
 'hpg_stores_lv2',
 'mean_lat_hpg_lv1',
 'max_lat_hpg_lv1',
 'min_lat_hpg_lv1',
 'mean_lon_hpg_lv1',
 'max_lon_hpg_lv1',
 'min_lon_hpg_lv1',
 'mean_lat_hpg_lv2',
 'max_lat_hpg_lv2',
 'min_lat_hpg_lv2',
 'mean_lon_hpg_lv2',
 'max_lon_hpg_lv2',
 'mi

In [388]:
full_data = pd.merge(full_data, air_reserve_grp, how='left', on=['air_store_id','visit_date'])
full_data = pd.merge(full_data, hpg_reserve_grp, how='left', on=['hpg_store_id','visit_date'])



In [389]:

list(full_data.columns.values)

['air_store_id',
 'id',
 'visit_date',
 'visitors',
 'dow',
 'year',
 'month',
 'doy',
 'dom',
 'woy',
 'is_month_end',
 'date_int',
 'air_genre_name',
 'air_area_name',
 'latitude_air',
 'longitude_air',
 'air_area_lv1',
 'air_area_lv2',
 'air_area_lv3',
 'air_stores_on_same_addr',
 'air_stores_lv1',
 'air_stores_lv2',
 'mean_lat_air_lv1',
 'max_lat_air_lv1',
 'min_lat_air_lv1',
 'mean_lon_air_lv1',
 'max_lon_air_lv1',
 'min_lon_air_lv1',
 'mean_lat_air_lv2',
 'max_lat_air_lv2',
 'min_lat_air_lv2',
 'mean_lon_air_lv2',
 'max_lon_air_lv2',
 'min_lon_air_lv2',
 'hpg_store_id',
 'hpg_genre_name',
 'hpg_area_name',
 'latitude_hpg',
 'longitude_hpg',
 'hpg_area_lv1',
 'hpg_area_lv2',
 'hpg_area_lv3',
 'hpg_stores_on_same_addr',
 'hpg_stores_lv1',
 'hpg_stores_lv2',
 'mean_lat_hpg_lv1',
 'max_lat_hpg_lv1',
 'min_lat_hpg_lv1',
 'mean_lon_hpg_lv1',
 'max_lon_hpg_lv1',
 'min_lon_hpg_lv1',
 'mean_lat_hpg_lv2',
 'max_lat_hpg_lv2',
 'min_lat_hpg_lv2',
 'mean_lon_hpg_lv2',
 'max_lon_hpg_lv2',
 'mi

In [390]:
list(date_info.columns.values)

['visit_date', 'day_of_week', 'holiday_flg']

In [391]:
date_info['visit_date'] = pd.to_datetime(date_info['visit_date'])
date_info['dow'] = date_info['visit_date'].dt.dayofweek

In [392]:
list(date_info.columns.values)

['visit_date', 'day_of_week', 'holiday_flg', 'dow']

In [393]:
date_info['date_len'] = len(date_info)

In [394]:
date_info.tail(20)

,visit_date,day_of_week,holiday_flg,dow,date_len
497,2017-05-12,Friday,0,4,517
498,2017-05-13,Saturday,0,5,517
499,2017-05-14,Sunday,0,6,517
500,2017-05-15,Monday,0,0,517
501,2017-05-16,Tuesday,0,1,517
502,2017-05-17,Wednesday,0,2,517
503,2017-05-18,Thursday,0,3,517
504,2017-05-19,Friday,0,4,517
505,2017-05-20,Saturday,0,5,517
506,2017-05-21,Sunday,0,6,517


In [395]:
date_info['weight'] = ((date_info.index + 1) / len(date_info)) ** 5 

In [396]:
date_info.head()

,visit_date,day_of_week,holiday_flg,dow,date_len,weight
0,2016-01-01,Friday,1,4,517,2.707368e-14
1,2016-01-02,Saturday,1,5,517,8.663577e-13
2,2016-01-03,Sunday,1,6,517,6.578904e-12
3,2016-01-04,Monday,0,0,517,2.772345e-11
4,2016-01-05,Tuesday,0,1,517,8.460525e-11


In [397]:
date_info['date_index'] = date_info.index + 1 
date_info['visit_date'] = date_info['visit_date'].dt.date

In [398]:
date_info.head()

,visit_date,day_of_week,holiday_flg,dow,date_len,weight,date_index
0,2016-01-01,Friday,1,4,517,2.707368e-14,1
1,2016-01-02,Saturday,1,5,517,8.663577e-13,2
2,2016-01-03,Sunday,1,6,517,6.578904e-12,3
3,2016-01-04,Monday,0,0,517,2.772345e-11,4
4,2016-01-05,Tuesday,0,1,517,8.460525e-11,5


In [399]:

list(full_data.columns.values)

['air_store_id',
 'id',
 'visit_date',
 'visitors',
 'dow',
 'year',
 'month',
 'doy',
 'dom',
 'woy',
 'is_month_end',
 'date_int',
 'air_genre_name',
 'air_area_name',
 'latitude_air',
 'longitude_air',
 'air_area_lv1',
 'air_area_lv2',
 'air_area_lv3',
 'air_stores_on_same_addr',
 'air_stores_lv1',
 'air_stores_lv2',
 'mean_lat_air_lv1',
 'max_lat_air_lv1',
 'min_lat_air_lv1',
 'mean_lon_air_lv1',
 'max_lon_air_lv1',
 'min_lon_air_lv1',
 'mean_lat_air_lv2',
 'max_lat_air_lv2',
 'min_lat_air_lv2',
 'mean_lon_air_lv2',
 'max_lon_air_lv2',
 'min_lon_air_lv2',
 'hpg_store_id',
 'hpg_genre_name',
 'hpg_area_name',
 'latitude_hpg',
 'longitude_hpg',
 'hpg_area_lv1',
 'hpg_area_lv2',
 'hpg_area_lv3',
 'hpg_stores_on_same_addr',
 'hpg_stores_lv1',
 'hpg_stores_lv2',
 'mean_lat_hpg_lv1',
 'max_lat_hpg_lv1',
 'min_lat_hpg_lv1',
 'mean_lon_hpg_lv1',
 'max_lon_hpg_lv1',
 'min_lon_hpg_lv1',
 'mean_lat_hpg_lv2',
 'max_lat_hpg_lv2',
 'min_lat_hpg_lv2',
 'mean_lon_hpg_lv2',
 'max_lon_hpg_lv2',
 'mi

In [400]:
full_data = pd.merge(full_data, date_info[['visit_date','date_len','date_index','weight', 'holiday_flg']], 
                     how='left', on='visit_date')

In [401]:

list(full_data.columns.values)

['air_store_id',
 'id',
 'visit_date',
 'visitors',
 'dow',
 'year',
 'month',
 'doy',
 'dom',
 'woy',
 'is_month_end',
 'date_int',
 'air_genre_name',
 'air_area_name',
 'latitude_air',
 'longitude_air',
 'air_area_lv1',
 'air_area_lv2',
 'air_area_lv3',
 'air_stores_on_same_addr',
 'air_stores_lv1',
 'air_stores_lv2',
 'mean_lat_air_lv1',
 'max_lat_air_lv1',
 'min_lat_air_lv1',
 'mean_lon_air_lv1',
 'max_lon_air_lv1',
 'min_lon_air_lv1',
 'mean_lat_air_lv2',
 'max_lat_air_lv2',
 'min_lat_air_lv2',
 'mean_lon_air_lv2',
 'max_lon_air_lv2',
 'min_lon_air_lv2',
 'hpg_store_id',
 'hpg_genre_name',
 'hpg_area_name',
 'latitude_hpg',
 'longitude_hpg',
 'hpg_area_lv1',
 'hpg_area_lv2',
 'hpg_area_lv3',
 'hpg_stores_on_same_addr',
 'hpg_stores_lv1',
 'hpg_stores_lv2',
 'mean_lat_hpg_lv1',
 'max_lat_hpg_lv1',
 'min_lat_hpg_lv1',
 'mean_lon_hpg_lv1',
 'max_lon_hpg_lv1',
 'min_lon_hpg_lv1',
 'mean_lat_hpg_lv2',
 'max_lat_hpg_lv2',
 'min_lat_hpg_lv2',
 'mean_lon_hpg_lv2',
 'max_lon_hpg_lv2',
 'mi

In [402]:
full_data.shape

(284127, 67)

In [403]:


full_data[full_data.columns[0:15]].head()

,air_store_id,id,visit_date,visitors,dow,year,month,doy,dom,woy,is_month_end,date_int,air_genre_name,air_area_name,latitude_air
0,air_ba937bf13d40fb24,NaN,2016-01-13,25,2,2016,1,13,31,2,False,20160113,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068
1,air_ba937bf13d40fb24,NaN,2016-01-14,32,3,2016,1,14,31,2,False,20160114,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068
2,air_ba937bf13d40fb24,NaN,2016-01-15,29,4,2016,1,15,31,2,False,20160115,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068
3,air_ba937bf13d40fb24,NaN,2016-01-16,22,5,2016,1,16,31,2,False,20160116,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068
4,air_ba937bf13d40fb24,NaN,2016-01-18,6,0,2016,1,18,31,3,False,20160118,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068


In [404]:
full_data[full_data.columns[15:30]].head()

,longitude_air,air_area_lv1,air_area_lv2,air_area_lv3,air_stores_on_same_addr,air_stores_lv1,air_stores_lv2,mean_lat_air_lv1,max_lat_air_lv1,min_lat_air_lv1,mean_lon_air_lv1,max_lon_air_lv1,min_lon_air_lv1,mean_lat_air_lv2,max_lat_air_lv2
0,139.751599,Tōkyō-to,Minato-ku,Shibakōen,51,444,61,35.675963,35.775664,35.546631,139.712946,139.868427,139.407843,35.660476,35.674918
1,139.751599,Tōkyō-to,Minato-ku,Shibakōen,51,444,61,35.675963,35.775664,35.546631,139.712946,139.868427,139.407843,35.660476,35.674918
2,139.751599,Tōkyō-to,Minato-ku,Shibakōen,51,444,61,35.675963,35.775664,35.546631,139.712946,139.868427,139.407843,35.660476,35.674918
3,139.751599,Tōkyō-to,Minato-ku,Shibakōen,51,444,61,35.675963,35.775664,35.546631,139.712946,139.868427,139.407843,35.660476,35.674918
4,139.751599,Tōkyō-to,Minato-ku,Shibakōen,51,444,61,35.675963,35.775664,35.546631,139.712946,139.868427,139.407843,35.660476,35.674918


In [405]:
full_data[full_data.columns[30:45]].head()

,min_lat_air_lv2,mean_lon_air_lv2,max_lon_air_lv2,min_lon_air_lv2,hpg_store_id,hpg_genre_name,hpg_area_name,latitude_hpg,longitude_hpg,hpg_area_lv1,hpg_area_lv2,hpg_area_lv3,hpg_stores_on_same_addr,hpg_stores_lv1,hpg_stores_lv2
0,35.658068,139.748928,139.751599,139.729426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,35.658068,139.748928,139.751599,139.729426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,35.658068,139.748928,139.751599,139.729426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,35.658068,139.748928,139.751599,139.729426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,35.658068,139.748928,139.751599,139.729426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [406]:
full_data[full_data.columns[45:60]].head()

,mean_lat_hpg_lv1,max_lat_hpg_lv1,min_lat_hpg_lv1,mean_lon_hpg_lv1,max_lon_hpg_lv1,min_lon_hpg_lv1,mean_lat_hpg_lv2,max_lat_hpg_lv2,min_lat_hpg_lv2,mean_lon_hpg_lv2,max_lon_hpg_lv2,min_lon_hpg_lv2,air_rvs,air_rv_dt_diff,mean_air_rv_dt_diff
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [407]:
full_data[full_data.columns[60: 75]].head()

,hpg_rvs,hpg_rv_dt_diff,mean_hpg_rv_dt_diff,date_len,date_index,weight,holiday_flg
0,NaN,NaN,NaN,517,13,1.005227e-08,0
1,NaN,NaN,NaN,517,14,1.456087e-08,0
2,NaN,NaN,NaN,517,15,2.055908e-08,0
3,NaN,NaN,NaN,517,16,2.838881e-08,0
4,NaN,NaN,NaN,517,18,5.115756e-08,0


In [408]:
full_data[full_data.columns[75:90]].head()

""
0
1
2
3
4


In [409]:

def result():
    i=0
    for (i<114): 
        print full_data[full_data.columns[i:15+i]].head() 
        i +=15

SyntaxError: invalid syntax (<ipython-input-409-aa66acad598e>, line 4)

In [ ]:
full_data[full_data.columns[90:105]].head()

In [ ]:
full_data[full_data.columns[105:120]].head()

In [ ]:
num_vars = ['dow', 'year', 'month', 'doy', 'dom', 'woy', 'holiday_flg',
            'date_index', 'date_int', 'air_stores_on_same_addr', 'hpg_stores_on_same_addr',
            'latitude_air', 'longitude_air', 
            'air_stores_on_same_addr', 'air_stores_lv1', 'air_stores_lv2',
            'mean_lat_air_lv1', 'max_lat_air_lv1', 'min_lat_air_lv1',
            'mean_lon_air_lv1', 'max_lon_air_lv1', 'min_lon_air_lv1',
            'mean_lat_air_lv2', 'max_lat_air_lv2', 'min_lat_air_lv2',
            'mean_lon_air_lv2', 'max_lon_air_lv2', 'min_lon_air_lv2',
            'latitude_hpg', 'longitude_hpg', 
            'hpg_stores_on_same_addr', 'hpg_stores_lv1', 'hpg_stores_lv2',
            'mean_lat_hpg_lv1', 'max_lat_hpg_lv1', 'min_lat_hpg_lv1',
            'mean_lon_hpg_lv1', 'max_lon_hpg_lv1', 'min_lon_hpg_lv1',
            'mean_lat_hpg_lv2', 'max_lat_hpg_lv2', 'min_lat_hpg_lv2',
            'mean_lon_hpg_lv2', 'max_lon_hpg_lv2', 'min_lon_hpg_lv2',
            'air_rvs', 'hpg_rvs','air_rv_dt_diff', 'hpg_rv_dt_diff']


cat_vars = ['air_store_id', 'air_genre_name', 'air_area_name', 'air_area_lv1', 'air_area_lv2', 'air_area_lv3',
            'hpg_store_id', 'hpg_genre_name', 'hpg_area_name', 'hpg_area_lv1', 'hpg_area_lv2', 'hpg_area_lv3']



In [ ]:
id_var = 'air_store_id'
target_var = 'visitors'

In [ ]:
LBL = preprocessing.LabelEncoder()

LE_vars=[]
LE_map=dict()

le = preprocessing.LabelEncoder()
>>> le.fit(["paris", "paris", "tokyo", "amsterdam"])
LabelEncoder()
>>> list(le.classes_)
['amsterdam', 'paris', 'tokyo']

In [ ]:
for cat_var in cat_vars:
    print ("Label Encoding %s" % (cat_var))
    LE_var=cat_var+'_le'
    full_data[LE_var]=LBL.fit_transform(full_data[cat_var].astype(str))
    LE_vars.append(LE_var)
    LE_map[cat_var]=LBL.classes_
    
print ("Label-encoded feaures: %s" % (LE_vars))

In [ ]:
LE_map

In [ ]:
LE_map[cat_var]

In [ ]:
LBL.classes_

In [ ]:
OHE = preprocessing.OneHotEncoder(sparse=True)
start=time.time()
OHE.fit(full_data[LE_vars])
OHE_sparse=OHE.transform(full_data[LE_vars])
                                   
print ('One-hot-encoding finished in %f seconds' % (time.time()-start))



In [ ]:
LE_vars

In [ ]:
OHE_sparse

In [ ]:
cat_vars

In [ ]:
OHE_vars = [var[:-3] + '_' + str(level).replace(' ','_')\
                for var in cat_vars for level in LE_map[var] ]

print ("OHE_sparse size :" ,OHE_sparse.shape)
print ("One-hot encoded catgorical feature samples : %s" % (OHE_vars[:100]))

In [ ]:
full_data['total_rvs'] = full_data['air_rvs'] + full_data['hpg_rvs']

In [ ]:
full_data['total_rvs'].tail()

In [ ]:
full_data['air_rvs'].head()

In [ ]:
full_data['hpg_rvs'].head()

In [ ]:
full_data['mean_rvs_air_hpg'] = full_data[['air_rvs','hpg_rvs']].apply(lambda x:np.mean(x), axis=1)
full_data['mean_dt_diff_air_hpg'] = full_data[['air_rv_dt_diff','hpg_rv_dt_diff']].apply(lambda x:np.mean(x), axis=1)



In [ ]:
full_data['longitude_air'].head()

In [ ]:
full_data['latitude_air'].head()

In [ ]:
full_data['lat_to_mean_lat_air_lv1'] = abs(full_data['latitude_air']-full_data['mean_lat_air_lv1'])
full_data['lat_to_max_lat_air_lv1']  = full_data['latitude_air']-full_data['max_lat_air_lv1']
full_data['lat_to_min_lat_air_lv1']  = full_data['latitude_air']-full_data['min_lat_air_lv1']
full_data['lon_to_mean_lon_air_lv1']  = abs(full_data['longitude_air']-full_data['mean_lon_air_lv1'])
full_data['lon_to_max_lon_air_lv1']  = full_data['longitude_air']-full_data['max_lon_air_lv1']
full_data['lon_to_min_lon_air_lv1']  = full_data['longitude_air']-full_data['min_lon_air_lv1']
full_data['lat_to_mean_lat_air_lv2'] = abs(full_data['latitude_air']-full_data['mean_lat_air_lv2'])
full_data['lat_to_max_lat_air_lv2']  = full_data['latitude_air']-full_data['max_lat_air_lv2']
full_data['lat_to_min_lat_air_lv2']  = full_data['latitude_air']-full_data['min_lat_air_lv2']
full_data['lon_to_mean_lon_air_lv2'] = abs(full_data['longitude_air']-full_data['mean_lon_air_lv2'])
full_data['lon_to_max_lon_air_lv2']  = full_data['longitude_air']-full_data['max_lon_air_lv2']
full_data['lon_to_min_lon_air_lv2']  = full_data['longitude_air']-full_data['min_lon_air_lv2']

full_data['lat_to_mean_lat_hpg_lv1'] = abs(full_data['latitude_hpg']-full_data['mean_lat_hpg_lv1'])
full_data['lat_to_max_lat_hpg_lv1']  = full_data['latitude_hpg']-full_data['max_lat_hpg_lv1']
full_data['lat_to_min_lat_hpg_lv1']  = full_data['latitude_hpg']-full_data['min_lat_hpg_lv1']
full_data['lon_to_mean_lon_hpg_lv1']  = abs(full_data['longitude_hpg']-full_data['mean_lon_hpg_lv1'])
full_data['lon_to_max_lon_hpg_lv1']  = full_data['longitude_hpg']-full_data['max_lon_hpg_lv1']
full_data['lon_to_min_lon_hpg_lv1']  = full_data['longitude_hpg']-full_data['min_lon_hpg_lv1']
full_data['lat_to_mean_lat_hpg_lv2'] = abs(full_data['latitude_hpg']-full_data['mean_lat_hpg_lv2'])
full_data['lat_to_max_lat_hpg_lv2']  = full_data['latitude_hpg']-full_data['max_lat_hpg_lv2']
full_data['lat_to_min_lat_hpg_lv2']  = full_data['latitude_hpg']-full_data['min_lat_hpg_lv2']
full_data['lon_to_mean_lon_hpg_lv2'] = abs(full_data['longitude_hpg']-full_data['mean_lon_hpg_lv2'])
full_data['lon_to_max_lon_hpg_lv2']  = full_data['longitude_hpg']-full_data['max_lon_hpg_lv2']
full_data['lon_to_min_lon_hpg_lv2']  = full_data['longitude_hpg']-full_data['min_lon_hpg_lv2']



In [ ]:
num_num_vars = ['total_rvs', 'mean_rvs_air_hpg',
       'mean_dt_diff_air_hpg', 'lon_plus_lat_air',
       'lat_to_mean_lat_air_lv1', 'lat_to_max_lat_air_lv1',
       'lat_to_min_lat_air_lv1', 'lon_to_mean_lon_air_lv1',
       'lon_to_max_lon_air_lv1', 'lon_to_min_lon_air_lv1',
       'lat_to_mean_lat_air_lv2', 'lat_to_max_lat_air_lv2',
       'lat_to_min_lat_air_lv2', 'lon_to_mean_lon_air_lv2',
       'lon_to_max_lon_air_lv2', 'lon_to_min_lon_air_lv2',
       'lat_to_mean_lat_hpg_lv1', 'lat_to_max_lat_hpg_lv1',
       'lat_to_min_lat_hpg_lv1', 'lon_to_mean_lon_hpg_lv1',
       'lon_to_max_lon_hpg_lv1', 'lon_to_min_lon_hpg_lv1',
       'lat_to_mean_lat_hpg_lv2', 'lat_to_max_lat_hpg_lv2',
       'lat_to_min_lat_hpg_lv2', 'lon_to_mean_lon_hpg_lv2',
       'lon_to_max_lon_hpg_lv2', 'lon_to_min_lon_hpg_lv2']

full_data.head()

In [ ]:
full_data['air_area_genre'] = full_data['air_area_name'] + '-' + full_data['air_genre_name'] 
full_data['air_store_dow'] = full_data['air_store_id'] + '-' + full_data['dow'].astype(str)
full_data['air_store_dow_holiday'] = full_data['air_store_id'] + '-' + full_data['dow'].astype(str) + '-' + full_data['holiday_flg'].astype(str)

cat_cat_vars = ['air_area_genre','air_store_dow', 'air_store_dow_holiday']

In [410]:
tmp = full_data[:train_size].groupby(['air_store_id','dow','holiday_flg'])['visitors'].\
            agg([np.mean, np.max, np.min, np.median]).\
            reset_index().\
            rename(columns={'mean':'mean_visitors',
                           'amax':'max_visitors',
                           'amin':'min_visitors',
                           'median':'median_visitors'})
            
full_data = pd.merge(full_data, tmp, how='left', on=['air_store_id','dow','holiday_flg'])

tmp = full_data[:train_size].groupby(['air_store_id','dow', 'holiday_flg']).\
            apply(lambda x:( (x.weight * x.visitors).sum() / x.weight.sum() )).\
            reset_index().rename(columns={0:'wmean_visitors'})
        
full_data = pd.merge(full_data, tmp, how='left', on=['air_store_id','dow','holiday_flg'])    
target_aggr_vars = ['mean_visitors', 'max_visitors', 'min_visitors', 'median_visitors', 'wmean_visitors']

In [411]:
full_data[:5]

,air_store_id,id,visit_date,visitors,dow,year,month,doy,dom,woy,...,mean_hpg_rv_dt_diff,date_len,date_index,weight,holiday_flg,mean_visitors,max_visitors,min_visitors,median_visitors,wmean_visitors
0,air_ba937bf13d40fb24,NaN,2016-01-13,25,2,2016,1,13,31,2,...,NaN,517,13,1.005227e-08,0,24.095238,57.0,7.0,25.0,18.793078
1,air_ba937bf13d40fb24,NaN,2016-01-14,32,3,2016,1,14,31,2,...,NaN,517,14,1.456087e-08,0,20.450000,45.0,7.0,21.0,15.205691
2,air_ba937bf13d40fb24,NaN,2016-01-15,29,4,2016,1,15,31,2,...,NaN,517,15,2.055908e-08,0,35.218750,61.0,17.0,35.5,33.938260
3,air_ba937bf13d40fb24,NaN,2016-01-16,22,5,2016,1,16,31,2,...,NaN,517,16,2.838881e-08,0,27.828125,53.0,6.0,27.0,25.022388
4,air_ba937bf13d40fb24,NaN,2016-01-18,6,0,2016,1,18,31,3,...,NaN,517,18,5.115756e-08,0,13.754386,34.0,2.0,12.0,9.326588


In [412]:
full_data[:4][target_var]

0    25
1    32
2    29
3    22
Name: visitors, dtype: int64

In [413]:
full_data[:4][target_var][:4].values

array([25, 32, 29, 22])

In [414]:
full_data[4:]

,air_store_id,id,visit_date,visitors,dow,year,month,doy,dom,woy,...,mean_hpg_rv_dt_diff,date_len,date_index,weight,holiday_flg,mean_visitors,max_visitors,min_visitors,median_visitors,wmean_visitors
4,air_ba937bf13d40fb24,NaN,2016-01-18,6,0,2016,1,18,31,3,...,NaN,517,18,5.115756e-08,0,13.754386,34.0,2.0,12.0,9.326588
5,air_ba937bf13d40fb24,NaN,2016-01-19,9,1,2016,1,19,31,3,...,NaN,517,19,6.703711e-08,0,18.580645,35.0,5.0,19.0,14.919559
6,air_ba937bf13d40fb24,NaN,2016-01-20,31,2,2016,1,20,31,3,...,NaN,517,20,8.663577e-08,0,24.095238,57.0,7.0,25.0,18.793078
7,air_ba937bf13d40fb24,NaN,2016-01-21,21,3,2016,1,21,31,3,...,NaN,517,21,1.105716e-07,0,20.450000,45.0,7.0,21.0,15.205691
8,air_ba937bf13d40fb24,NaN,2016-01-22,18,4,2016,1,22,31,3,...,NaN,517,22,1.395278e-07,0,35.218750,61.0,17.0,35.5,33.938260
9,air_ba937bf13d40fb24,NaN,2016-01-23,26,5,2016,1,23,31,3,...,NaN,517,23,1.742555e-07,0,27.828125,53.0,6.0,27.0,25.022388
10,air_ba937bf13d40fb24,NaN,2016-01-25,21,0,2016,1,25,31,4,...,NaN,517,25,2.643914e-07,0,13.754386,34.0,2.0,12.0,9.326588
11,air_ba937bf13d40fb24,NaN,2016-01-26,11,1,2016,1,26,31,4,...,NaN,517,26,3.216726e-07,0,18.580645,35.0,5.0,19.0,14.919559
12,air_ba937bf13d40fb24,NaN,2016-01-27,24,2,2016,1,27,31,4,...,NaN,517,27,3.884777e-07,0,24.095238,57.0,7.0,25.0,18.793078
13,air_ba937bf13d40fb24,NaN,2016-01-28,21,3,2016,1,28,31,4,...,NaN,517,28,4.659480e-07,0,20.450000,45.0,7.0,21.0,15.205691


In [415]:
full_data

,air_store_id,id,visit_date,visitors,dow,year,month,doy,dom,woy,...,mean_hpg_rv_dt_diff,date_len,date_index,weight,holiday_flg,mean_visitors,max_visitors,min_visitors,median_visitors,wmean_visitors
0,air_ba937bf13d40fb24,NaN,2016-01-13,25,2,2016,1,13,31,2,...,NaN,517,13,1.005227e-08,0,24.095238,57.0,7.0,25.0,18.793078
1,air_ba937bf13d40fb24,NaN,2016-01-14,32,3,2016,1,14,31,2,...,NaN,517,14,1.456087e-08,0,20.450000,45.0,7.0,21.0,15.205691
2,air_ba937bf13d40fb24,NaN,2016-01-15,29,4,2016,1,15,31,2,...,NaN,517,15,2.055908e-08,0,35.218750,61.0,17.0,35.5,33.938260
3,air_ba937bf13d40fb24,NaN,2016-01-16,22,5,2016,1,16,31,2,...,NaN,517,16,2.838881e-08,0,27.828125,53.0,6.0,27.0,25.022388
4,air_ba937bf13d40fb24,NaN,2016-01-18,6,0,2016,1,18,31,3,...,NaN,517,18,5.115756e-08,0,13.754386,34.0,2.0,12.0,9.326588
5,air_ba937bf13d40fb24,NaN,2016-01-19,9,1,2016,1,19,31,3,...,NaN,517,19,6.703711e-08,0,18.580645,35.0,5.0,19.0,14.919559
6,air_ba937bf13d40fb24,NaN,2016-01-20,31,2,2016,1,20,31,3,...,NaN,517,20,8.663577e-08,0,24.095238,57.0,7.0,25.0,18.793078
7,air_ba937bf13d40fb24,NaN,2016-01-21,21,3,2016,1,21,31,3,...,NaN,517,21,1.105716e-07,0,20.450000,45.0,7.0,21.0,15.205691
8,air_ba937bf13d40fb24,NaN,2016-01-22,18,4,2016,1,22,31,3,...,NaN,517,22,1.395278e-07,0,35.218750,61.0,17.0,35.5,33.938260
9,air_ba937bf13d40fb24,NaN,2016-01-23,26,5,2016,1,23,31,3,...,NaN,517,23,1.742555e-07,0,27.828125,53.0,6.0,27.0,25.022388


In [417]:
full_data[4:][4:]

,air_store_id,id,visit_date,visitors,dow,year,month,doy,dom,woy,...,mean_hpg_rv_dt_diff,date_len,date_index,weight,holiday_flg,mean_visitors,max_visitors,min_visitors,median_visitors,wmean_visitors
8,air_ba937bf13d40fb24,NaN,2016-01-22,18,4,2016,1,22,31,3,...,NaN,517,22,1.395278e-07,0,35.218750,61.0,17.0,35.5,33.938260
9,air_ba937bf13d40fb24,NaN,2016-01-23,26,5,2016,1,23,31,3,...,NaN,517,23,1.742555e-07,0,27.828125,53.0,6.0,27.0,25.022388
10,air_ba937bf13d40fb24,NaN,2016-01-25,21,0,2016,1,25,31,4,...,NaN,517,25,2.643914e-07,0,13.754386,34.0,2.0,12.0,9.326588
11,air_ba937bf13d40fb24,NaN,2016-01-26,11,1,2016,1,26,31,4,...,NaN,517,26,3.216726e-07,0,18.580645,35.0,5.0,19.0,14.919559
12,air_ba937bf13d40fb24,NaN,2016-01-27,24,2,2016,1,27,31,4,...,NaN,517,27,3.884777e-07,0,24.095238,57.0,7.0,25.0,18.793078
13,air_ba937bf13d40fb24,NaN,2016-01-28,21,3,2016,1,28,31,4,...,NaN,517,28,4.659480e-07,0,20.450000,45.0,7.0,21.0,15.205691
14,air_ba937bf13d40fb24,NaN,2016-01-29,26,4,2016,1,29,31,4,...,NaN,517,29,5.553123e-07,0,35.218750,61.0,17.0,35.5,33.938260
15,air_ba937bf13d40fb24,NaN,2016-01-30,6,5,2016,1,30,31,4,...,NaN,517,30,6.578904e-07,0,27.828125,53.0,6.0,27.0,25.022388
16,air_ba937bf13d40fb24,NaN,2016-02-03,18,2,2016,2,34,29,5,...,NaN,517,34,1.230104e-06,0,24.095238,57.0,7.0,25.0,18.793078
17,air_ba937bf13d40fb24,NaN,2016-02-04,12,3,2016,2,35,29,5,...,NaN,517,35,1.421960e-06,0,20.450000,45.0,7.0,21.0,15.205691


In [419]:
num_num_vars = ['total_rvs', 'mean_rvs_air_hpg',
       'mean_dt_diff_air_hpg', 'lon_plus_lat_air',
       'lat_to_mean_lat_air_lv1', 'lat_to_max_lat_air_lv1',
       'lat_to_min_lat_air_lv1', 'lon_to_mean_lon_air_lv1',
       'lon_to_max_lon_air_lv1', 'lon_to_min_lon_air_lv1',
       'lat_to_mean_lat_air_lv2', 'lat_to_max_lat_air_lv2',
       'lat_to_min_lat_air_lv2', 'lon_to_mean_lon_air_lv2',
       'lon_to_max_lon_air_lv2', 'lon_to_min_lon_air_lv2',
       'lat_to_mean_lat_hpg_lv1', 'lat_to_max_lat_hpg_lv1',
       'lat_to_min_lat_hpg_lv1', 'lon_to_mean_lon_hpg_lv1',
       'lon_to_max_lon_hpg_lv1', 'lon_to_min_lon_hpg_lv1',
       'lat_to_mean_lat_hpg_lv2', 'lat_to_max_lat_hpg_lv2',
       'lat_to_min_lat_hpg_lv2', 'lon_to_mean_lon_hpg_lv2',
       'lon_to_max_lon_hpg_lv2', 'lon_to_min_lon_hpg_lv2']



In [421]:
LBL = preprocessing.LabelEncoder()

cat_cat_LE_vars=[]
cat_cat_LE_map=dict()
for cat_var in cat_cat_vars:
    print ("Label Encoding %s" % (cat_var))
    LE_var=cat_var+'_le'
    full_data[LE_var]=LBL.fit_transform(full_data[cat_var].astype(str))
    cat_cat_LE_vars.append(LE_var)
    cat_cat_LE_map[cat_var]=LBL.classes_
    
print ("Label-encoded feaures: %s" % (cat_cat_LE_vars))

NameError: name 'cat_cat_vars' is not defined

In [420]:
full_vars = num_vars + LE_vars + num_num_vars + cat_cat_LE_vars + target_aggr_vars
    
train = full_data[:train_size]
y = full_data[:train_size][target_var][:train_size].values
test = full_data[train_size:]
ids = full_data[train_size:][train_size:].id.values

print ('train data size:', train.shape, 'test data size:', test.shape)    

NameError: name 'cat_cat_LE_vars' is not defined